In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [5]:
df_categories = pd.read_csv('steam_data/categories.csv')
df_games = pd.read_csv('steam_data/games.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")
df_genres = pd.read_csv('steam_data/genres.csv')
df_reviews = pd.read_csv('steam_data/reviews.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")
df_tags = pd.read_csv('steam_data/tags.csv')
df_steamspy_insights = pd.read_csv('steam_data/steamspy_insights.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")

/var/folders/kl/f84_8w7x3kg1ccbk5n2lqjg40000gn/T/ipykernel_2545/574882810.py:4: DtypeWarning: Columns (1,3,4,5,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv('steam_data/reviews.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")


# Pre-processing

Converting json-like object in price_overview column into separate column for easier usage

In [6]:
def extract_price_info(price_str):
    try:
        data = json.loads(price_str)
    except (TypeError, json.JSONDecodeError):
        return pd.Series({
            'final': None,
            'initial': None,
            'currency': None,
            'final_formatted': None,
            'discount_percent': None,
            'initial_formatted': None,
        })
    return pd.Series({
        'final': data.get('final'),
        'initial': data.get('initial'),
        'currency': data.get('currency'),
        'final_formatted': data.get('final_formatted'),
        'discount_percent': data.get('discount_percent'),
        'initial_formatted': data.get('initial_formatted'),
    })

In [7]:
price_details = df_games['price_overview'].apply(extract_price_info)
df_games = pd.concat([df_games, price_details], axis=1)

# to remvoe the original 'price_overview' column
df_games = df_games.drop(columns=['price_overview'])

Deleting inconsistant data

In [8]:
df_games.drop(columns=['languages'], inplace=True)
df_reviews.drop(columns=['reviews'], inplace=True)
df_steamspy_insights.drop(columns=['languages'], inplace=True)

In [9]:
df_selected_games = df_games[['app_id', 'name', 'release_date', 'is_free', 'type']]
df_selected_steamspy = df_steamspy_insights[['app_id', 'developer', 'publisher', 'owners_range']]
df_selected_reviews = df_reviews[['app_id','review_score', 'review_score_description', 'positive', 'negative', 'total']]

df_combined = pd.merge(df_selected_games, df_selected_steamspy, on='app_id', how='inner')
df_combined = pd.merge(df_combined, df_selected_reviews, on='app_id', how='inner')

In [10]:
df_combined.head()

,app_id,name,release_date,is_free,type,developer,publisher,owners_range,review_score,review_score_description,positive,negative,total
0,10,Counter-Strike,2000-11-01,0,game,Valve,Valve,"10,000,000 .. 20,000,000",9,Overwhelmingly Positive,235403,6207,241610
1,20,Team Fortress Classic,1999-04-01,0,game,Valve,Valve,"5,000,000 .. 10,000,000",8,Very Positive,7315,1094,8409
2,30,Day of Defeat,2003-05-01,0,game,Valve,Valve,"5,000,000 .. 10,000,000",8,Very Positive,6249,672,6921
3,40,Deathmatch Classic,2001-06-01,0,game,Valve,Valve,"5,000,000 .. 10,000,000",8,Very Positive,2542,524,3066
4,50,Half-Life: Opposing Force,1999-11-01,0,game,Gearbox Software,Valve,"2,000,000 .. 5,000,000",9,Overwhelmingly Positive,22263,1111,23374


In [ ]:
df_categories.to_csv('steam_data_fixed/df_categories.csv', index=False)
df_genres.to_csv('steam_data_fixed/df_genres.csv', index=False)
df_reviews.to_csv('steam_data_fixed/df_reviews.csv', index=False)
df_tags.to_csv('steam_data_fixed/df_tags.csv', index=False)
df_steamspy_insights.to_csv('steam_data_fixed/df_steamspy_insights.csv', index=False)
df_games.to_csv('steam_data_fixed/games.csv', index=False)